# 📊 Análise Exploratória de Dados (TikTok)
Autor: **Paulo Everton**

Explorar o dataset dos arquivos do meu perfil pessoal no tiktok.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
df_tiktok = pd.read_csv("dataset/Overview.csv", encoding="utf-8-sig")

In [ ]:
print("Primeiras linhas do dataset:")
display(df_tiktok.head())

print("Últimas linhas do dataset:")
display(df_tiktok.tail())

print("\nInformações gerais sobre o dataset:")
print(df_tiktok.info())

print("\nQuantidade de linhas e colunas (shape):")
print(df_tiktok.shape)

In [ ]:
print("Estatísticas descritivas das colunas numéricas:")
display(df_tiktok.describe())

In [ ]:
print("Quantidade de valores nulos por coluna:")
print(df_tiktok.isnull().sum())

print("\nPercentual de valores nulos (%):")
print((df_tiktok.isnull().sum() / len(df_tiktok) * 100).round(2))

In [ ]:
#tentando converter colunas que pareçam datas
for col in df_tiktok.columns:
    if "Date" in col:
        df_tiktok[col] = pd.to_datetime(df_tiktok[col], errors='coerce')

print("Tipos de dados das colunas:")
print(df_tiktok.dtypes)

In [ ]:
print('Total de curtidas:', df_tiktok['Likes'].sum())
print('Total de comentários:', df_tiktok['Comments'].sum())
print('Total de compartilhamentos:', df_tiktok['Shares'].sum())
print('Total de visualizações em vídeos:', df_tiktok['Video Views'].sum())
print('Total de visualizações no perfil:', df_tiktok['Profile Views'].sum())

In [ ]:
df_likes = df_tiktok['Likes'].sum()
df_comments = df_tiktok['Comments'].sum()
df_shares = df_tiktok['Shares'].sum()
df_video = df_tiktok['Video Views'].sum()
df_profile = df_tiktok['Profile Views'].sum()

dados = {
    'Likes': df_likes,
    'Comments': df_comments,
    'Shares': df_shares,
    'Video Views': df_video,
    'Profile Views': df_profile
}

plt.figure(figsize=(10, 6))

barras = plt.barh(list(dados.keys()), list(dados.values()), color=sns.color_palette('husl'))

for barra in barras:
    plt.text(barra.get_width() + max(dados.values()) * 0.01,
             barra.get_y() + barra.get_height() / 2,
             f'{int(barra.get_width()):,}',
             va='center', fontsize=10)
    
plt.title('Desempenho Total no TikTok', fontsize=14, fontweight='bold')
plt.xlabel('Quantidade Total', fontsize=12)
plt.ylabel('Métrica', fontsize=12)
plt.box(False)
plt.tight_layout()
plt.show()

In [ ]:
print("Quantidade de valores únicos por coluna:")
for col in df_tiktok.columns:
    print(f"{col}: {df_tiktok[col].nunique()} valores únicos")

In [ ]:
df_formated = df_tiktok[['Video Views', 'Profile Views', 'Likes', 'Comments', 'Shares']]

df_formated.hist(figsize=(12,8), bins=20)
plt.suptitle("Distribuição das Variáveis Numéricas", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df_tiktok.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlação entre as Variáveis Numéricas")
plt.show()

In [ ]:
for col in df_tiktok.select_dtypes(include='number'):
    plt.figure(figsize=(5, 3))
    sns.boxplot(x=df_tiktok[col])
    plt.title(f'Boxplot da coluna: {col}')
    plt.tight_layout()
    plt.show()

In [ ]:
df_tiktok['Date'] = pd.to_datetime(df_tiktok['Date'])

In [ ]:
df_tiktok['Engajamento Total'] = df_tiktok['Likes'] + df_tiktok['Comments'] + df_tiktok['Shares']
df_tiktok['Taxa de Engajamento'] = df_tiktok['Engajamento Total'] / df_tiktok['Video Views'].replace(0, np.nan)
df_tiktok['Taxa de Visita ao Perfil'] = df_tiktok['Profile Views'] / df_tiktok['Video Views'].replace(0, np.nan)
df_tiktok['Taxa de Compartilhamento'] = df_tiktok['Shares'] / df_tiktok['Video Views'].replace(0, np.nan)

In [ ]:
dias_sem_engajamento = (df_tiktok['Engajamento Total'] == 0).sum()
print(f"Dias sem engajamento: {dias_sem_engajamento}")

In [ ]:
df_tiktok = df_tiktok.sort_values('Date')

df_continuo = df_tiktok.set_index('Date').asfreq('D').fillna(0)

plt.figure(figsize=(12,6))
plt.plot(df_continuo.index, df_continuo['Video Views'], label='Visualizações de Vídeo')
plt.plot(df_continuo.index, df_continuo['Engajamento Total'], label='Engajamento Total')
plt.title('Visualizações e Engajamento ao Longo do Ano (Eixo Contínuo)')
plt.xlabel('Data')
plt.ylabel('Quantidade')
plt.legend()
plt.show()


In [ ]:
df_weekly = df_tiktok.set_index('Date').resample('W').mean(numeric_only=True)

plt.figure(figsize=(12,6))
plt.plot(df_weekly.index, df_weekly['Video Views'], label='Visualizações de Vídeo')
plt.plot(df_weekly.index, df_weekly['Engajamento Total'], label='Engajamento Total')
plt.title('Média Semanal - Visualizações e Engajamento')
plt.legend()
plt.show()

In [ ]:
df_growth = df_tiktok.set_index('Date')[['Video Views', 'Likes', 'Comments', 'Shares']].pct_change()

plt.figure(figsize=(15, 6))

for coluna in df_growth.columns:
    plt.plot(df_growth.index, df_growth[coluna], marker='o', linewidth=2, label=coluna)
    plt.fill_between(df_growth.index, df_growth[coluna], alpha=0.2)

plt.title('Taxa de Crescimento Diário (%) — Dados Filtrados', fontsize=14, fontweight='bold')
plt.xlabel('Data', fontweight='bold')
plt.ylabel('% de Variação', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.legend(title='Métricas', fontsize=10)
plt.tight_layout()

plt.show()

In [ ]:
df_tiktok['weekday_num'] = df_tiktok['Date'].dt.weekday  #0 = Monday, 6 = Sunday
df_tiktok['day_name'] = df_tiktok['Date'].dt.day_name()  #nome do dia no idioma do ambiente

# Ordena os nomes dos dias pelo número do weekday
ordered_days = df_tiktok[['weekday_num', 'day_name']].drop_duplicates().sort_values('weekday_num')['day_name'].tolist()

plt.figure(figsize=(10,6))
sns.boxplot(x='day_name', y='Engajamento Total', data=df_tiktok, order=ordered_days)
plt.title('Engajamento Total por Dia da Semana')
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_tiktok['Engajamento Relativo (%)'] = (
    df_tiktok['Engajamento Total'] / df_tiktok['Engajamento Total'].mean()
) * 100

plt.figure(figsize=(12,5))
plt.plot(df_tiktok['Date'], df_tiktok['Engajamento Relativo (%)'])
plt.axhline(100, color='red', linestyle='--', label='Média')
plt.title('Engajamento Relativo Diário (%)')
plt.legend()
plt.show()

In [ ]:
df_tiktok['Média Móvel Engajamento (7d)'] = df_tiktok['Engajamento Total'].rolling(window=7).mean()
df_tiktok['Média Móvel Visualizações (7d)'] = df_tiktok['Video Views'].rolling(window=7).mean()

plt.figure(figsize=(12,6))
plt.plot(df_tiktok['Date'], df_tiktok['Média Móvel Engajamento (7d)'], label='Engajamento (7 dias)')
plt.plot(df_tiktok['Date'], df_tiktok['Média Móvel Visualizações (7d)'], label='Visualizações (7 dias)')
plt.title('Médias Móveis de Engajamento e Visualizações (7 dias)')
plt.legend()
plt.show()

In [ ]:
df_tiktok['Crescimento Acumulado Views (%)'] = (
    df_tiktok['Video Views'].cumsum() / df_tiktok['Video Views'].sum()
) * 100

plt.figure(figsize=(10,5))
plt.plot(df_tiktok['Date'], df_tiktok['Crescimento Acumulado Views (%)'])
plt.title('Crescimento Acumulado das Visualizações (%)')
plt.ylabel('% Acumulado')
plt.show()